In [1]:
import json
import requests
from requests.exceptions import HTTPError
import pandas as pd
from collections import defaultdict, Counter

In [10]:
def fetch_orcid_data(url: str) -> dict:
  """
  Fetches publication data from the ORCID API.
  """
  try:
    response = requests.get(url)
    response.raise_for_status()
    return response.json()

  except HTTPError as http_err:
      raise HTTPError(f'Connection error: {http_err}')

  except requests.exceptions.RequestException as req_err:
      raise requests.exceptions.RequestException(f'API request failed: {req_err}')


def aggregate_group(group: dict) -> dict:
  """
  Aggregates multiple ORCID work versions inside one group.
  Returns single publication record.
  """
  work=[]

  # Собираем все описания для работы
  for w in group.get('works', []):
    title = (w.get('title') or {}).get('value')
    year = (w.get('publicationDate') or {}).get('year', 0)
    doi = None
    eid = None

    for i in w.get('workExternalIdentifiers', []):
      id_type = (i.get('externalIdentifierType') or {}).get('value')
      number = (i.get('externalIdentifierId') or {}).get('value')

      if id_type == 'doi' and number:
        doi = number.strip().lower()
      if id_type == 'eid' and number:
        eid = number.strip()
    work.append({
        'year': int(year),
        'title': title,
        'doi': doi,
        'eid': eid
    })

  if not work:
    return None

  # Приоритетной считаем информацию о работе с максимальным годом (либо если присутствует в любой версии)
  best = max(work, key=lambda x: x['year'] or 0)

  best['title'] = best['title'] or next((i['title'] for i in work if i['title']), None)
  best['doi'] = best['doi'] or next((i['doi'] for i in work if i['doi']), None)
  best['eid'] = best['eid'] or next((i['eid'] for i in work if i['eid']), None)

  return best

def build_publications(data: dict) -> pd.DataFrame:
  """
  Builds a consolidated DataFrame of publications from ORCID data.
  """
  publications = []

  for group in data.get('groups', []):
    record = aggregate_group(group)
    if record:
      publications.append(record)

  df = pd.DataFrame(publications)
  df = df.sort_values(by='year', ascending=False).reset_index(drop=True)
  return df

In [11]:
url = 'https://orcid.org/0000-0002-8520-7267/allWorks.json?sort=date&sortAsc=false'

data = fetch_orcid_data(url)
works_df = build_publications(data)

works_df.head()

,year,title,doi,eid
0,2026,Resource Allocation Schemes for Scalable Panel...,10.1109/ojvt.2026.3652908,None
1,2026,Deep Unfolding of Iterative Precoding-Based Nu...,10.1109/tcomm.2025.3642692,None
2,2025,Multi-Eavesdropper Detection Through PHY-Aware...,10.1109/access.2025.3598502,2-s2.0-105013310122
3,2025,Sequential Hybrid Beamforming for Radio Stripes,10.1109/access.2025.3571771,None
4,2025,Nonlinear Massive MIMO–RSMA With Multilinear G...,10.1109/access.2025.3638959,None


In [13]:
works_df.to_excel('report.xlsx', index=False)